In [1]:
from google.colab import drive
drive.mount('/content/gdrive/', force_remount=True)

Mounted at /content/gdrive/


In [ ]:
!ls gdrive/MyDrive/datasets/ham10000

HAM10000_images_part_1	HAM10000_images_part_2	HAM10000_metadata.csv


In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
from glob import glob
import seaborn as sns
from PIL import Image
np.random.seed(0)
from sklearn.preprocessing import label_binarize
from sklearn.metrics import confusion_matrix
import itertools

In [3]:
import torch
import torchvision.datasets
from torchvision import transforms

In [4]:
X_train = torch.load('gdrive/MyDrive/datasets/ham_100_X_train.pt')
y_train = torch.load('gdrive/MyDrive/datasets/ham_100_y_train.pt')
X_test = torch.load('gdrive/MyDrive/datasets/ham_100_X_test.pt')
y_test = torch.load('gdrive/MyDrive/datasets/ham_100_y_test.pt')

In [ ]:
X_train = torch.nn.functional.normalize(X_train)
X_test = torch.nn.functional.normalize(X_test)

In [ ]:
X_train /= 255.
X_test /= 255.

In [ ]:
X_train.shape, y_train.shape

(torch.Size([8012, 224, 224, 3]), torch.Size([8012]))

In [ ]:
X_train.shape, y_train.shape

(torch.Size([8012, 224, 224, 3]), torch.Size([8012]))

In [5]:
X_train = X_train.permute(0, 3, 1, 2)
X_test = X_test.permute(0, 3, 1, 2)

In [6]:
import random

import torchvision.datasets
from tqdm import tqdm

random.seed(0)
np.random.seed(0)
torch.manual_seed(0)
torch.cuda.manual_seed(0)
torch.backends.cudnn.deterministic = True

In [7]:
import sys
sys.path.insert(0, '/content/gdrive/MyDrive/Colab Notebooks')

In [8]:
import pytorch_optimizer as optim

In [38]:
def train(net, X_train, y_train, X_test, y_test):
    device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
    net = net.to(device)
    loss = torch.nn.CrossEntropyLoss()
    #optimizer = torch.optim.SGD(net.parameters(), lr=5.0e-2)
    #optimizer = torch.optim.Adam(net.parameters(), lr=1.0e-3)
    #optimizer = optim.PNMBelief(net.parameters(), lr=0.001)
    #optimizer = optim.YogiPNM(net.parameters())
    #optimizer = optim.AdaBelief(net.parameters())
    optimizer = optim.DiffGrad(net.parameters())
    #optimizer = optim.Yogi(net.parameters())
    
    batch_size = 25

    test_accuracy_history = []
    test_loss_history = []

    X_test = X_test.to(device)
    y_test = y_test.to(device)

    for epoch in range(14):
        print("epoch ", epoch)
        order = np.random.permutation(len(X_train))
        
        if epoch == 9:
            #optimizer = torch.optim.Adam(net.parameters(), lr=1.0e-5)
            #optimizer = optim.PNMBelief(net.parameters(), lr=1.0e-4)
            #optimizer = optim.YogiPNM(net.parameters(), lr=1.0e-4)
            #optimizer = optim.AdaBelief(net.parameters(), lr=1.0e-4)
            optimizer = optim.DiffGrad(net.parameters(), lr=1.0e-4)
            #optimizer = optim.Yogi(net.parameters(), lr=1.0e-4)
    

        for start_index in tqdm(range(0, len(X_train), batch_size)):
            optimizer.zero_grad()
            net.train()

            batch_indexes = order[start_index:start_index+batch_size]

            X_batch = X_train[batch_indexes].to(device)
            y_batch = y_train[batch_indexes].to(device)

            preds = net.forward(X_batch)

            loss_value = loss(preds, y_batch)
            loss_value.backward()

            optimizer.step()
            
            X_batch

        net.eval()
        with torch.no_grad():
            test_preds = net.forward(X_test)
            loss_value = loss(test_preds, y_test).item()
            test_loss_history.append(loss_value)

            accuracy = (test_preds.argmax(dim=1) == y_test).float().mean().item()
            test_accuracy_history.append(accuracy)
            #print(test_preds.argmax(dim=1))
            #print(y_test)

        print("accuracy = ", accuracy)
        print("\n\n")

    return test_accuracy_history, test_loss_history

In [10]:
accuracies = {}
losses = {}

In [11]:
from torchvision.models import densenet121, DenseNet121_Weights

Downloading: "https://download.pytorch.org/models/densenet121-a639ec97.pth" to /root/.cache/torch/hub/checkpoints/densenet121-a639ec97.pth
100%|██████████| 30.8M/30.8M [00:00<00:00, 95.6MB/s]


In [12]:
model = densenet121(weights=DenseNet121_Weights.IMAGENET1K_V1)
accuracies['densenet121+Adam'], losses['densenet121+Adam'] = \
    train(model, X_train, y_train, X_test, y_test)

epoch  0


100%|██████████| 321/321 [00:34<00:00,  9.20it/s]


accuracy =  0.7289066314697266



epoch  1


100%|██████████| 321/321 [00:25<00:00, 12.42it/s]


accuracy =  0.7738392353057861



epoch  2


...

accuracy =  0.8177733421325684



epoch  12


100%|██████████| 321/321 [00:26<00:00, 12.28it/s]


accuracy =  0.8267598748207092



epoch  13


100%|██████████| 321/321 [00:25<00:00, 12.43it/s]


accuracy =  0.8307538628578186





In [23]:
model = densenet121(weights=DenseNet121_Weights.IMAGENET1K_V1)
accuracies['densenet121+PNMBelief'], losses['densenet121+PNMBelief'] = \
    train(model, X_train, y_train, X_test, y_test)

epoch  0


100%|██████████| 321/321 [00:43<00:00,  7.33it/s]


accuracy =  0.7838242650032043



epoch  1


100%|██████████| 321/321 [00:44<00:00,  7.17it/s]


accuracy =  0.7943085432052612



epoch  2


...

accuracy =  0.8457313776016235



epoch  12


100%|██████████| 321/321 [00:43<00:00,  7.32it/s]


accuracy =  0.8427358865737915



epoch  13


100%|██████████| 321/321 [00:43<00:00,  7.30it/s]


accuracy =  0.8427358865737915





In [31]:
model = densenet121(weights=DenseNet121_Weights.IMAGENET1K_V1)
accuracies['densenet121+Yogi'], losses['densenet121+Yogi'] = \
    train(model, X_train, y_train, X_test, y_test)

epoch  0


100%|██████████| 321/321 [00:34<00:00,  9.25it/s]


accuracy =  0.7763354778289795



epoch  1


100%|██████████| 321/321 [00:35<00:00,  9.10it/s]


accuracy =  0.8077883124351501



epoch  2


...

accuracy =  0.8542186617851257



epoch  12


100%|██████████| 321/321 [00:36<00:00,  8.79it/s]


accuracy =  0.8512231707572937



epoch  13


100%|██████████| 321/321 [00:35<00:00,  9.01it/s]


accuracy =  0.8572141528129578





In [39]:
model = densenet121(weights=DenseNet121_Weights.IMAGENET1K_V1)
accuracies['densenet121+DiffGrad'], losses['densenet121+DiffGrad'] = \
    train(model, X_train, y_train, X_test, y_test)

epoch  0


100%|██████████| 321/321 [00:47<00:00,  6.70it/s]


accuracy =  0.781328022480011



epoch  1


100%|██████████| 321/321 [00:48<00:00,  6.60it/s]


accuracy =  0.7758362293243408



epoch  2


...

accuracy =  0.8302546143531799



epoch  12


100%|██████████| 321/321 [00:47<00:00,  6.70it/s]


accuracy =  0.8397403955459595



epoch  13


100%|██████████| 321/321 [00:48<00:00,  6.65it/s]


accuracy =  0.8382426500320435





# ansamble

In [13]:
APreds = pd.DataFrame()

In [40]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
X_test = X_test.to(device)
y_test = y_test.to(device)
test_preds = []
model.eval()
with torch.no_grad():
            test_preds = model.forward(X_test)

In [15]:
APreds['Adam'] = test_preds.argmax(dim=1).tolist()

In [25]:
APreds['PNMBelief'] = test_preds.argmax(dim=1).tolist()

In [33]:
APreds['Yogi'] = test_preds.argmax(dim=1).tolist()

In [41]:
APreds['DiffGrad'] = test_preds.argmax(dim=1).tolist()

In [42]:
APreds

,Adam,PNMBelief,Yogi,DiffGrad
0,4,4,4,4
1,4,4,4,4
2,4,4,4,4
3,2,4,4,4
4,0,4,4,4
...,...,...,...,...
1998,4,4,4,4
1999,4,5,5,5
2000,4,4,4,4
2001,4,4,4,4


In [ ]:
APreds = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/ansambl_ham10000_densenet.csv')
APreds = APreds.drop(['Unnamed: 0'], axis=1)

In [ ]:
APreds.to_csv('/content/gdrive/MyDrive/Colab Notebooks/ansambl_ham10000_densenet.csv')

In [47]:
ARes = pd.DataFrame(np.zeros((2003,7), dtype=int))

for i in range(2003):
    for j in range(4):
        if j == 0:
            ARes.iloc[i, APreds.iloc[i, j]] += 0.83075
        if j == 1:
            ARes.iloc[i, APreds.iloc[i, j]] += 0.8427
        if j == 2:
            ARes.iloc[i, APreds.iloc[i, j]] += 0.85721
        if j == 3:
            ARes.iloc[i, APreds.iloc[i, j]] += 0.83824

In [48]:
(torch.tensor(ARes.idxmax(axis=1)).to(device) == y_test).float().mean().item()

0.8617074489593506

In [ ]:
data = np.zeros((7,7), dtype=int)
D = pd.DataFrame(data)

for x, y in zip(test_preds.argmax(dim=1).tolist(), y_test.tolist()):
    D[y][x] += 1

In [ ]:
D #Adam

In [ ]:
D #PNMBelief

In [ ]:
D #Yogi

In [ ]:
D #DiffGrad

In [ ]:
data = np.zeros((7,7), dtype=int)
D = pd.DataFrame(data)

for x, y in zip(ARes.idxmax(axis=1).to_list(), y_test.tolist()):
    D[y][x] += 1

In [ ]:
D #ansamble

In [ ]:
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

In [ ]:
precision_score(y_test, test_preds.argmax(dim=1), average='micro')


In [ ]:
print('precision: ', precision_score(y_test, test_preds.argmax(dim=1), average=None))
print('recall: ', recall_score(y_test, test_preds.argmax(dim=1), average=None))
print('f1_score: ', f1_score(y_test, test_preds.argmax(dim=1), average=None))

# SoftMax

In [17]:
ARes2 = pd.DataFrame(np.zeros((2003,7), dtype=int))

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
X_test = X_test.to(device)
y_test = y_test.to(device)
test_preds = []
model.eval()
with torch.no_grad():
            test_preds = model.forward(X_test)

In [26]:
m = torch.nn.Softmax(dim=1)

In [19]:
test_preds = m(test_preds) * 0.83075 #Adam

In [27]:
test_preds = m(test_preds) * 0.8427 #PNMBelief

In [35]:
test_preds = m(test_preds) * 0.85721 #Yogi

In [43]:
test_preds = m(test_preds) * 0.83824 #DiffGrad

In [44]:
ARes2 += pd.DataFrame(test_preds.to('cpu'))

In [ ]:
test_preds

In [45]:
ARes2

,0,1,2,3,4,5,6
0,6.535642e-08,1.873217e-06,0.000003,9.074580e-07,3.368413,0.000002,2.724990e-06
1,3.354090e-06,4.694088e-07,0.000594,2.467791e-07,3.363643,0.004639,5.628572e-07
2,3.627682e-07,2.648599e-08,0.000011,4.481307e-07,3.215877,0.153011,3.823239e-07
3,1.211849e-02,3.194410e-05,0.772018,6.093672e-04,2.582428,0.001685,7.082423e-06
4,7.893076e-01,2.470300e-02,0.025013,7.821424e-03,2.520138,0.000066,9.232403e-05
...,...,...,...,...,...,...,...
1998,6.093685e-09,1.286543e-08,0.000006,2.625478e-07,3.367310,0.001583,6.525512e-07
1999,1.121823e-05,1.073568e-04,0.000995,1.809383e-05,0.821548,2.545989,2.225405e-04
2000,2.506618e-04,5.159609e-05,0.022281,2.598389e-03,3.343084,0.000009,2.312572e-05
2001,1.223928e-06,6.482592e-06,0.000189,1.165022e-04,3.367252,0.001323,1.098666e-05


In [46]:
(torch.tensor(ARes2.idxmax(axis=1)).to(device) == y_test).float().mean().item()

0.8612082004547119